In [2]:
import gurobipy as gp #using gurobi for optimization
from gurobipy import GRB
import pandas as pd

#create data mappings for table headers based on data files in GitHub repo

#mapping manufacturing locations to IDs
mfg_name_to_id = {'CAM': 1, 'LEV': 2, 'LETH': 3}
id_to_mfg_name = {v: k for k, v in mfg_name_to_id.items()}
MANUFACTURERS = list(mfg_name_to_id.keys())

#mapping DC locations to IDs
dc_name_to_id = {'CORN': 1, 'SUR': 2, 'CAL': 3, 'VAU': 4}
id_to_dc_name = {v: k for k, v in dc_name_to_id.items()}
DCS = list(dc_name_to_id.keys())

#mapping store locations to IDs
store_name_to_id = {'MONT': 1, 'NEW': 2, 'KEL': 3}
id_to_store_name = {v: k for k, v in store_name_to_id.items()}
STORES = list(store_name_to_id.keys())

#mapping Product 1 and Product 2 to IDs
product_name_to_id = {'Product 1': 1, 'Product 2': 2}
id_to_product_name = {v: k for k, v in product_name_to_id.items()}
PRODUCTS = list(product_name_to_id.keys())

#output summary of above mapping sets to verify that ID mapping was done correctly
print("Mapping Results")
print(f"Manufacturer Mappings: {mfg_name_to_id}")
print(f"DC Mappings: {dc_name_to_id}")
print(f"Store Mappings: {store_name_to_id}")
print(f"Product Mappings: {product_name_to_id}\n")

#read CSVs of the data tables (in Github repo)

#manufacturing production cost (c_ij): cost to produce item i at manufacturer j
df_production_cost = pd.read_csv('cost_mfg_product.csv')
production_cost = {}
for index, row in df_production_cost.iterrows():
    mfg_id = mfg_name_to_id.get(row['MFG'])
    for col in ['Product 1', 'Product 2']:
        product_id = product_name_to_id.get(col)
        if mfg_id is not None and product_id is not None:
            production_cost[(product_id, mfg_id)] = row[col]

#manufacturing capacity (cap_j): max production quantity for each manufacturer j
manufacturer_capacity = {
    mfg_name_to_id['CAM']: 151000,
    mfg_name_to_id['LEV']: 91000,
    mfg_name_to_id['LETH']: 61000,
}

#DC capacity (cap_k): max throughput for each DC k
dc_capacity = {
    dc_name_to_id['CORN']: 111000,
    dc_name_to_id['SUR']: 95000,
    dc_name_to_id['CAL']: 63000,
    dc_name_to_id['VAU']: 48000,
}

#store Capacity (cap_e): max amount of inventory space in stores e
store_capacity = {
    store_name_to_id['MONT']: 75000,
    store_name_to_id['NEW']: 77000,
    store_name_to_id['KEL']: 61000,
}

#fixed costs from manufacturer to DC (fixed_manu_dc_jk)
df_fixed_manu_dc = pd.read_csv('fixedcost_manu_to_dc.csv')
fixed_manu_dc_cost = {}
df_fixed_manu_dc.rename(columns={df_fixed_manu_dc.columns[0]: 'DC_Name'}, inplace=True)
for index, row in df_fixed_manu_dc.iterrows():
    dc_id = dc_name_to_id.get(row['DC_Name'])
    for col_name in df_fixed_manu_dc.columns[1:]:
        mfg_id = mfg_name_to_id.get(col_name)
        if mfg_id is not None and dc_id is not None:
            fixed_manu_dc_cost[(mfg_id, dc_id)] = row[col_name]

#variable cost manufacturer to DC (var_manu_dc_ijk)
var_manu_dc_cost = {}
for prod_name, prod_id in product_name_to_id.items():
    prod_file_suffix = 'prod1.csv' if prod_name == 'Product 1' else 'prod2.csv'
    df_var_manu_dc = pd.read_csv(f'variablecost_manu_to_dc_{prod_file_suffix}')
    df_var_manu_dc.rename(columns={df_var_manu_dc.columns[0]: 'DC_Name'}, inplace=True)
    for index, row in df_var_manu_dc.iterrows():
        dc_id = dc_name_to_id.get(row['DC_Name'])
        for col_name in df_var_manu_dc.columns[1:]:
            mfg_id = mfg_name_to_id.get(col_name)
            if dc_id is not None and mfg_id is not None:
                var_manu_dc_cost[(prod_id, mfg_id, dc_id)] = row[col_name]


#fixed costs from DC to store (fixed_dc_store_ke)
df_fixed_dc_store = pd.read_csv('fixedcost_dc_to_store.csv')
fixed_dc_store_cost = {}
df_fixed_dc_store.rename(columns={df_fixed_dc_store.columns[0]: 'Store_Name'}, inplace=True)
for index, row in df_fixed_dc_store.iterrows():
    store_id = store_name_to_id.get(row['Store_Name'])
    for col_name in df_fixed_dc_store.columns[1:]:
        dc_id = dc_name_to_id.get(col_name)
        if store_id is not None and dc_id is not None:
            fixed_dc_store_cost[(dc_id, store_id)] = row[col_name]


#variable cost from DC to store (var_dc_store_ike)
var_dc_store_cost = {}
for prod_name, prod_id in product_name_to_id.items():
    prod_file_suffix = 'prod1.csv' if prod_name == 'Product 1' else 'prod2.csv'
    df_var_dc_store = pd.read_csv(f'variablecost_dc_to_store_{prod_file_suffix}')
    df_var_dc_store.rename(columns={df_var_dc_store.columns[0]: 'Store_Name'}, inplace=True)
    for index, row in df_var_dc_store.iterrows():
        store_id = store_name_to_id.get(row['Store_Name'])
        for col_name in df_var_dc_store.columns[1:]:
            dc_id = dc_name_to_id.get(col_name)
            if store_id is not None and dc_id is not None:
                var_dc_store_cost[(prod_id, dc_id, store_id)] = row[col_name]

#store demand data (demand_ie)
df_store_demand = pd.read_csv('store_demand_data.csv')
store_demand = {}
for index, row in df_store_demand.iterrows():
    store_id = store_name_to_id.get(row['Store_ID'])
    if store_id is not None:
        store_demand[(product_name_to_id['Product 1'], store_id)] = row['Product 1 Demand']
        store_demand[(product_name_to_id['Product 2'], store_id)] = row['Product 2 Demand']

#Big M value to help with linking binary variables (route open or closed) to continuous route ijk flow variables
BIG_M = 1000000

print("Data Parameters from CSV files\n")

#create opti model
model = gp.Model("optimodel")

#define decision variables

#x_ij = quantity of product i produced by manufacturer j
x_ij = model.addVars(product_name_to_id.values(), mfg_name_to_id.values(), name="production", vtype=GRB.CONTINUOUS, lb=0.0)

#trans_ijk = quantity of product i shipped from manufacturer j to DC k
trans_ijk = model.addVars(product_name_to_id.values(), mfg_name_to_id.values(), dc_name_to_id.values(), name="manu_to_dc_flow", vtype=GRB.CONTINUOUS, lb=0.0)

#trans_ike = quantity of product i shipped from DC k to store e
trans_ike = model.addVars(product_name_to_id.values(), dc_name_to_id.values(), store_name_to_id.values(), name="dc_to_store_flow", vtype=GRB.CONTINUOUS, lb=0.0)

#y_jk = binary variable where 1 if route from manufacturer j to DC k is open, 0 otherwise
y_jk = model.addVars(mfg_name_to_id.values(), dc_name_to_id.values(), name="manu_dc_route_open", vtype=GRB.BINARY)

#z_ke = binary variable where 1 if route from DC k to store e is open, 0 otherwise
z_ke = model.addVars(dc_name_to_id.values(), store_name_to_id.values(), name="dc_store_route_open", vtype=GRB.BINARY)

#EXTENSION: NEW BINARY DECISION VARIABLES FOR PROMOTIONS
promo_i = model.addVars(product_name_to_id.values(), vtype=GRB.BINARY, name=f"promo")

#output decision variables
print("Decision Variables\n")

#set objective function to [Total Cost = Production Cost + Fixed MFG-DC Cost + Fixed DC-Store Cost + Variable MFG-DC Transport Cost + Variable DC-Store Transport Cost]

#production cost - sum over all products i and manufacturers j: x_ij * production_cost[i,j]
obj_production_cost = gp.quicksum(
    x_ij[prod_id, mfg_id] * production_cost[prod_id, mfg_id]
    for prod_id in product_name_to_id.values() for mfg_id in mfg_name_to_id.values()
)

#fixed cost for manufacturing to DC routes - sum over all manufacturers j and DCs k: fixed_manu_dc_cost[j,k] * y_jk[j,k]
obj_fixed_manu_dc_cost = gp.quicksum(
    fixed_manu_dc_cost[mfg_id, dc_id] * y_jk[mfg_id, dc_id]
    for mfg_id in mfg_name_to_id.values() for dc_id in dc_name_to_id.values()
)

#fixed cost for DC to store routes - sum over all DCs k and stores e: fixed_dc_store_cost[k,e] * z_ke[k,e]
obj_fixed_dc_store_cost = gp.quicksum(
    fixed_dc_store_cost[dc_id, store_id] * z_ke[dc_id, store_id]
    for dc_id in dc_name_to_id.values() for store_id in store_name_to_id.values()
)

#variable transportation cost manufacturing to DC - sum over all products i, manufacturers j, and DCs k: var_manu_dc_cost[i,j,k] * trans_ijk[i,j,k]
obj_variable_manu_dc_cost = gp.quicksum(
    var_manu_dc_cost[prod_id, mfg_id, dc_id] * trans_ijk[prod_id, mfg_id, dc_id]
    for prod_id in product_name_to_id.values() for mfg_id in mfg_name_to_id.values() for dc_id in dc_name_to_id.values()
)

#variable transportation cost DC to store - sum over all products i, DCs k, and stores e: var_dc_store_cost[i,k,e] * trans_ike[i,k,e]
obj_variable_dc_store_cost = gp.quicksum(
    var_dc_store_cost[prod_id, dc_id, store_id] * trans_ike[prod_id, dc_id, store_id]
    for prod_id in product_name_to_id.values() for dc_id in dc_name_to_id.values() for store_id in store_name_to_id.values()
)

#EXTENSION: PROMOTION PRICES AND COSTS FOR OBJECTIVE FUNCTION
promo_price_factor = [0.19, 0.22] 
product_price = [15,40]
promo_fixed_cost = [50000,133333]

#EXTENSION: REVENUE CALCULATION
#use normal price if no promotion, or discounted price if promotion is active
revenue = gp.quicksum((1 - promo_i[prod_id]) * product_price[prod_id-1] * gp.quicksum(trans_ike[prod_id, dc_id, store_id] for dc_id in dc_name_to_id.values() for store_id in store_name_to_id.values()) +
    promo_i[prod_id] * (1 - promo_price_factor[prod_id-1]) * product_price[prod_id-1] * gp.quicksum(trans_ike[prod_id, dc_id, store_id] for dc_id in dc_name_to_id.values() for store_id in store_name_to_id.values())
    for prod_id in product_name_to_id.values())

#EXTENSION: TOTAL PROMOTION FIXED COSTS
promo_fixed_cost_total = gp.quicksum(promo_fixed_cost[prod_id-1] * promo_i[prod_id] for prod_id in product_name_to_id.values())

#EXTENSION: ADD REVENUE AND PROMOTION COSTS TO OBJECTIVE FUNCTION
model.setObjective(
    revenue -
    (obj_production_cost +
    obj_fixed_manu_dc_cost +
    obj_fixed_dc_store_cost +
    obj_variable_manu_dc_cost +
    obj_variable_dc_store_cost + 
    promo_fixed_cost_total),
    GRB.MAXIMIZE
)

#print objective function set
print("Objective Function Set to Maximize Total Profit\n")

#add constraints to model

#EXTENSION: PROMOTION DEMAND INCREASE FACTORS
promo_demand_factor = [0.63,0.7]

#Constraint 1: manufacturing production capacity constraints; total quantity of all products made by manufacturer must be <= capacity
#EXTENSION: ADD 40% CAPACITY IF PROMOTION IS ACTIVE
for mfg_id in mfg_name_to_id.values():
    model.addConstr(
        gp.quicksum(x_ij[prod_id, mfg_id] for prod_id in product_name_to_id.values()) <= (1 + 0.4 * promo_i[prod_id]) * manufacturer_capacity[mfg_id],
        f"ManuCapacity_{id_to_mfg_name[mfg_id]}"
    )

#Constraint 2: the total amount of product i made by maunfacturer j must be equal to the total amount of product i shipped from manufacturer j to all DCs (flow)
for prod_id in product_name_to_id.values():
    for mfg_id in mfg_name_to_id.values():
        model.addConstr(
            x_ij[prod_id, mfg_id] == gp.quicksum(trans_ijk[prod_id, mfg_id, dc_id] for dc_id in dc_name_to_id.values()),
            f"ManuOutputFlow_{id_to_product_name[prod_id]}_{id_to_mfg_name[mfg_id]}"
        )

#Constraint 3: DC capacity constraint; total quantity of products flowing into DC must be <= capacity
#EXTENSION: ADD 20% CAPACITY IF PROMOTION IS ACTIVE
for dc_id in dc_name_to_id.values():
    model.addConstr(
        gp.quicksum(trans_ijk[prod_id, mfg_id, dc_id] for prod_id in product_name_to_id.values() for mfg_id in mfg_name_to_id.values()) <= (1 + 0.2 * promo_i[prod_id]) * dc_capacity[dc_id],
        f"DCInboundCapacity_{id_to_dc_name[dc_id]}"
    )

#Constraint 4: the total amount of product i flowing into DCs must be equal to the total amount of product i shipped from the DCs to the stores (flow)
for prod_id in product_name_to_id.values():
    for dc_id in dc_name_to_id.values():
        model.addConstr(
            gp.quicksum(trans_ijk[prod_id, mfg_id, dc_id] for mfg_id in mfg_name_to_id.values()) == gp.quicksum(trans_ike[prod_id, dc_id, store_id] for store_id in store_name_to_id.values()),
            f"DCFlowBalance_{id_to_product_name[prod_id]}_{id_to_dc_name[dc_id]}"
        )

#Constraint 5: the total quantity of products received at each store must be at least equal to the store's demand
#EXTENSION: APPLY DEMAND INCREASE FACTOR IF PROMOTION IS ACTIVE
for prod_id in product_name_to_id.values():
    for store_id in store_name_to_id.values():
        model.addConstr(
            gp.quicksum(trans_ike[prod_id, dc_id, store_id] for dc_id in dc_name_to_id.values()) >= (1 + promo_demand_factor[prod_id-1] * promo_i[prod_id]) * store_demand[prod_id, store_id],
            f"StoreDemand_{id_to_product_name[prod_id]}_{id_to_store_name[store_id]}"
        )

#Constraint 6: the total quantity of products received at each store must be <= to its capacity
#EXTENSION: APPLY DEMAND INCREASE FACTOR IF PROMOTION IS ACTIVE FOR CAPACITY
for store_id in store_name_to_id.values():
    model.addConstr(
        gp.quicksum(trans_ike[prod_id, dc_id, store_id] for prod_id in product_name_to_id.values() for dc_id in dc_name_to_id.values()) <= (1 + promo_demand_factor[prod_id-1] * promo_i[prod_id]) *store_capacity[store_id],
        f"StoreCapacityInbound_{id_to_store_name[store_id]}"
    )

#Constraint 7: If a manufacturer-DC route (j, k) is not open (y_jk = 0), then no product can flow through it and if it is open (y_jk = 1), then products can flow into it
for prod_id in product_name_to_id.values():
    for mfg_id in mfg_name_to_id.values():
        for dc_id in dc_name_to_id.values():
            model.addConstr(
                trans_ijk[prod_id, mfg_id, dc_id] <= BIG_M * y_jk[mfg_id, dc_id], #using Big M to activate/de-activate this constraint based on if flow allowed
                f"Link_ManuDC_Flow_{id_to_product_name[prod_id]}_{id_to_mfg_name[mfg_id]}_{id_to_dc_name[dc_id]}"
            )

#Constraint 8: similar to Constraint 7, but for DC-Store routes instead
for prod_id in product_name_to_id.values():
    for dc_id in dc_name_to_id.values():
        for store_id in store_name_to_id.values():
            model.addConstr(
                trans_ike[prod_id, dc_id, store_id] <= BIG_M * z_ke[dc_id, store_id], #using Big M to activate/de-activate this constraint based on if flow allowed
                f"Link_DCStore_Flow_{id_to_product_name[prod_id]}_{id_to_dc_name[dc_id]}_{id_to_store_name[store_id]}"
            )

#output constraints
print("Model Constraints\n")

#optimize the model
model.optimize()

#print optimization model results with objective value and routes
print("\n--- Optimization Results ---")

if model.status == GRB.OPTIMAL:
    print(f"Optimal Total Profit: ${model.objVal:,.2f}\n")

    print("\n--- Promotion Decisions ---")
    for prod_id in product_name_to_id.values():
        if promo_i[prod_id].x > 0.5:
            print(f"  {id_to_product_name[prod_id]} is PROMOTED")
        else:
            print(f"  {id_to_product_name[prod_id]} is NOT PROMOTED")

    print("--- Production by Manufacturer (x_ij) ---")
    for prod_id in product_name_to_id.values():
        for mfg_id in mfg_name_to_id.values():
            if x_ij[prod_id, mfg_id].x > 1e-6: # Check for non-zero production
                print(f"  {id_to_product_name[prod_id]} produced by {id_to_mfg_name[mfg_id]}: {x_ij[prod_id, mfg_id].x:,.2f} units")

    print("\n--- Manufacturer-DC Routes Open (y_jk) ---")
    for mfg_id in mfg_name_to_id.values():
        for dc_id in dc_name_to_id.values():
            if y_jk[mfg_id, dc_id].x > 0.5: # Check if binary variable is essentially 1
                print(f"  Route from {id_to_mfg_name[mfg_id]} to {id_to_dc_name[dc_id]} is OPEN")

    print("\n--- Flow from Manufacturers to DCs (trans_ijk) ---")
    for prod_id in product_name_to_id.values():
        for mfg_id in mfg_name_to_id.values():
            for dc_id in dc_name_to_id.values():
                if trans_ijk[prod_id, mfg_id, dc_id].x > 1e-6: # Check for non-zero flow
                    print(f"  {id_to_product_name[prod_id]} from {id_to_mfg_name[mfg_id]} to {id_to_dc_name[dc_id]}: {trans_ijk[prod_id, mfg_id, dc_id].x:,.2f} units")

    print("\n--- DC-Store Routes Open (z_ke) ---")
    for dc_id in dc_name_to_id.values():
        for store_id in store_name_to_id.values():
            if z_ke[dc_id, store_id].x > 0.5: # Check if binary variable is essentially 1
                print(f"  Route from {id_to_dc_name[dc_id]} to {id_to_store_name[store_id]} is OPEN")

    print("\n--- Flow from DCs to Stores (trans_ike) ---")
    for prod_id in product_name_to_id.values():
        for dc_id in dc_name_to_id.values():
            for store_id in store_name_to_id.values():
                if trans_ike[prod_id, dc_id, store_id].x > 1e-6: # Check for non-zero flow
                    print(f"  {id_to_product_name[prod_id]} from {id_to_dc_name[dc_id]} to {id_to_store_name[store_id]}: {trans_ike[prod_id, dc_id, store_id].x:,.2f} units")

elif model.status == GRB.INFEASIBLE:
    print("The model is INFEASIBLE. No solution satisfies all constraints.")
    print("Consider reviewing your data and constraints for contradictions.")
elif model.status == GRB.UNBOUNDED:
    print("The model is UNBOUNDED. The objective can be arbitrarily improved.")
    print("This often means a necessary constraint or bound is missing.")
else:
    print(f"Optimization ended with status: {model.status}")

Mapping Results
Manufacturer Mappings: {'CAM': 1, 'LEV': 2, 'LETH': 3}
DC Mappings: {'CORN': 1, 'SUR': 2, 'CAL': 3, 'VAU': 4}
Store Mappings: {'MONT': 1, 'NEW': 2, 'KEL': 3}
Product Mappings: {'Product 1': 1, 'Product 2': 2}

Data Parameters from CSV files

Decision Variables

Objective Function Set to Maximize Total Profit

Model Constraints

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.5.0 24F74)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 78 rows, 80 columns and 268 nonzeros
Model fingerprint: 0x9d6a3004
Model has 24 quadratic objective terms
Variable types: 54 continuous, 26 integer (26 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e-01, 1e+05]
  QObjective range [6e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+04, 2e+05]
Found heuristic solution: objective 4297114.0194
Presolve remo